In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import datetime

from sklearn.utils import shuffle
import matplotlib.pyplot as plt


from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing


In [2]:
Train = pd.read_csv("train.csv", index_col="id")
Test = pd.read_csv("test.csv", index_col="id")
Display= pd.read_csv("train.csv", index_col="id")

In [3]:

Train.describe()

,playtime_forever,price,total_positive_reviews,total_negative_reviews
count,357.000000,3.570000e+02,355.000000,355.000000
mean,3.119234,7.123277e+04,15356.115493,3348.991549
std,11.213114,9.228827e+05,44032.358308,24706.625113
min,0.000000,0.000000e+00,0.000000,0.000000
25%,0.000000,3.600000e+03,799.000000,115.500000
50%,0.083333,6.800000e+03,2809.000000,437.000000
75%,1.616667,9.000000e+03,9666.500000,1220.500000
max,113.800000,1.599990e+07,440902.000000,436046.000000


In [4]:
def exttdate(df,col):
    df[col+'_year'] = pd.to_datetime(df[col]).apply(lambda x:x.year)
    df[col+'_month'] = pd.to_datetime(df[col]).apply(lambda x:x.month)
    df[col+'_day'] = pd.to_datetime(df[col]).apply(lambda x:x.day)
    df[col+'_weekday'] = pd.to_datetime(df[col]).apply(lambda x:x.dayofweek)
    
    
    
    
    

In [5]:
Train = Train[Train.purchase_date.notnull()]

In [6]:
#Train['month'] = pd.DatetimeIndex((pd.to_datetime(Train['purchase_date']))).month
exttdate(Train,'purchase_date')
exttdate(Test,'purchase_date')

Train['datediff'] = (pd.to_datetime(Train['purchase_date'])-pd.to_datetime(Train['release_date'])).dt.days
Train['prepurchase'] = ((pd.to_datetime(Train['purchase_date'])-pd.to_datetime(Train['release_date'])).dt.days <=0)
#Test['month'] = pd.DatetimeIndex((pd.to_datetime(Test['purchase_date']))).month
Test['datediff'] = (pd.to_datetime(Train['purchase_date'])-pd.to_datetime(Test['release_date'])).dt.days
Test['prepurchase'] = ((pd.to_datetime(Test['purchase_date'])-pd.to_datetime(Test['release_date'])).dt.days <=0)

In [7]:
Train['ifplay']= (Train['playtime_forever']>0)

In [8]:
Train

,playtime_forever,is_free,price,genres,categories,tags,purchase_date,release_date,total_positive_reviews,total_negative_reviews,purchase_date_year,purchase_date_month,purchase_date_day,purchase_date_weekday,datediff,prepurchase,ifplay
id,,,,,,,,,,,,,,,,,
0,0.000000,False,3700.0,"Adventure,Casual,Indie","Single-player,Steam Trading Cards,Steam Cloud","Indie,Adventure,Story Rich,Casual,Atmospheric,...","Jul 2, 2018","10 Dec, 2013",372.0,96.0,2018,7,2,0,1665,False,False
1,0.016667,True,0.0,RPG,"Single-player,Partial Controller Support","Mod,Utilities,RPG,Game Development,Singleplaye...","Nov 26, 2016","12 Aug, 2015",23.0,0.0,2016,11,26,5,472,False,True
2,0.000000,False,5000.0,"Adventure,Casual,Indie","Single-player,Full controller support,Steam Tr...","Point & Click,Adventure,Story Rich,Comedy,Indi...","Jul 2, 2018","28 Jan, 2014",3018.0,663.0,2018,7,2,0,1616,False,False
3,1.533333,False,9900.0,"Action,RPG","Single-player,Multi-player,Steam Achievements,...","Medieval,RPG,Open World,Strategy,Sandbox,Actio...","Nov 28, 2016","31 Mar, 2010",63078.0,1746.0,2016,11,28,0,2434,False,True
4,22.333333,False,4800.0,"Action,Indie,Strategy","Single-player,Co-op,Steam Achievements,Full co...","Tower Defense,Co-op,Action,Strategy,Online Co-...","Mar 4, 2018","30 Jul, 2012",8841.0,523.0,2018,3,4,6,2043,False,True
6,2.616667,False,3600.0,"Action,Indie","Single-player,Steam Achievements,Steam Trading...","Great Soundtrack,Action,Violent,Indie,Top-Down...","Feb 16, 2018","23 Oct, 2012",39776.0,1035.0,2018,2,16,4,1942,False,True
7,0.150000,True,0.0,Simulation,"Single-player,Multi-player,Online Multi-Player...","VR,Simulation,Utilities,Multiplayer","Jan 29, 2018","28 Apr, 2016",33.0,3.0,2018,1,29,0,641,False,True
8,0.016667,False,4000.0,"Adventure,RPG","Single-player,Steam Achievements,Steam Trading...","RPG,Turn-Based,Adventure,Fantasy,Great Soundtr...","Nov 23, 2018","25 Feb, 2014",1875.0,39.0,2018,11,23,4,1732,False,True
9,0.016667,False,8800.0,"Action,Adventure","Single-player,Steam Achievements,SteamVR Colle...","Action,Adventure,VR,Shooter,Puzzle,FPS,First-P...","Dec 22, 2017","17 Oct, 2017",47.0,11.0,2017,12,22,4,66,False,True


In [9]:
#target encoding
def add_noise(series, noise_level):
    return series * (1 + noise_level * np.random.randn(len(series)))

def target_encode(trn_series=None, 
                  tst_series=None, 
                  target=None, 
                  min_samples_leaf=1, 
                  smoothing=1,
                  noise_level=0):
    """
    Smoothing is computed like in the following paper by Daniele Micci-Barreca
    trn_series : training categorical feature as a pd.Series
    tst_series : test categorical feature as a pd.Series
    target : target data as a pd.Series
    min_samples_leaf (int) : minimum samples to take category average into account
    smoothing (int) : smoothing effect to balance categorical average vs prior  
    """ 
    assert len(trn_series) == len(target)
    assert trn_series.name == tst_series.name
    temp = pd.concat([trn_series, target], axis=1)
    # Compute target mean 
    averages = temp.groupby(by=trn_series.name)[target.name].agg(["mean", "count"])
    # Compute smoothing
    smoothing = 1 / (1 + np.exp(-(averages["count"] - min_samples_leaf) / smoothing))
    # Apply average function to all target data
    prior = target.mean()
    # The bigger the count the less full_avg is taken into account
    averages[target.name] = prior * (1 - smoothing) + averages["mean"] * smoothing
    averages.drop(["mean", "count"], axis=1, inplace=True)
    # Apply averages to trn and tst series
    ft_trn_series = pd.merge(
        trn_series.to_frame(trn_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=trn_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    # pd.merge does not keep the index so restore it
    ft_trn_series.index = trn_series.index 
    ft_tst_series = pd.merge(
        tst_series.to_frame(tst_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=tst_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    # pd.merge does not keep the index so restore it
    ft_tst_series.index = tst_series.index
    return add_noise(ft_trn_series, noise_level), add_noise(ft_tst_series, noise_level)

In [10]:
valid_fraction=0.1
valid_size=int(len(Train)*valid_fraction)

dicttrain = Train
#dicttrain = dicttrain[dicttrain.jname.notnull()]

test=dicttrain[-valid_size:]
train = dicttrain

# Target encode year
trainyear, testyear = target_encode(train["purchase_date_year"],
                         test["purchase_date_year"], 
                         target=train['playtime_forever'], 
                         min_samples_leaf=10,
                         smoothing=10,
                         noise_level=0)
train = pd.concat([train,trainyear],axis =1)
#test = pd.concat([test,testjname],axis =1)
# Target encode tname
# traintname, testtname = target_encode(train["tname"],
#                          test["tname"], 
#                          target=train['rank'], 
#                          min_samples_leaf=100,
#                          smoothing=10,
#                          noise_level=0.01)
# train = pd.concat([train,traintname],axis =1)
# test = pd.concat([test,testtname],axis =1)
# Target encode horsenum
trainmonth, testmonth = target_encode(train["purchase_date_month"],
                              test["purchase_date_month"], 
                              target=train['playtime_forever'], 
                              min_samples_leaf=10,
                              smoothing=10,
                              noise_level=0)
train = pd.concat([train,trainmonth],axis =1)

trainday, testday = target_encode(train["purchase_date_day"],
                              test["purchase_date_day"], 
                              target=train['playtime_forever'], 
                              min_samples_leaf=10,
                              smoothing=10,
                              noise_level=0)
train = pd.concat([train,trainday],axis =1)

trainwday, testwday = target_encode(train["purchase_date_weekday"],
                              test["purchase_date_weekday"], 
                              target=train['playtime_forever'], 
                              min_samples_leaf=10,
                              smoothing=10,
                              noise_level=0)
train = pd.concat([train,trainwday],axis =1)

#test = pd.concat([test,testhorsenum],axis =1)

year_dict = dict(zip(train.purchase_date_year, train.purchase_date_year_mean))
month_dict = dict(zip(train.purchase_date_month, train.purchase_date_month_mean))
day_dict = dict(zip(train.purchase_date_day, train.purchase_date_day_mean))
wday_dict = dict(zip(train.purchase_date_weekday, train.purchase_date_weekday_mean))


In [11]:
Test['purchase_date_year_mean'] = Test['purchase_date_year'].map(year_dict)
Test['purchase_date_month_mean']= Test['purchase_date_month'].map(month_dict)
Test['purchase_date_day_mean'] = Test['purchase_date_day'].map(day_dict)
Test['purchase_date_weekday_mean']= Test['purchase_date_weekday'].map(wday_dict)

In [12]:
wday_dict

{0: 5.265664077738344,
 5: 1.70843902804054,
 6: 8.214651806776322,
 4: 3.6582027225937632,
 1: 1.08791788369863,
 2: 0.8133066958801553,
 3: 1.8647864229593218}

In [13]:
dropcol=['purchase_date_year','purchase_date_month','purchase_date_day','purchase_date_weekday']

In [14]:
train =train.drop(dropcol,axis = 1)
Test =Test.drop(dropcol,axis = 1)

In [15]:
#Train['playtime_forever'].value_counts()

In [16]:

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
print("Setup Complete")

Setup Complete


In [17]:
train = shuffle(train)
Y_train = train['playtime_forever']
Y2_train =train['ifplay']
X_train = train.drop(['playtime_forever','ifplay'],axis = 1)
#X_train = train.drop(['ifplay'],axis = 1)
#X_train=Train

In [18]:
X_train.head(100)

,is_free,price,genres,categories,tags,purchase_date,release_date,total_positive_reviews,total_negative_reviews,datediff,prepurchase,purchase_date_year_mean,purchase_date_month_mean,purchase_date_day_mean,purchase_date_weekday_mean
id,,,,,,,,,,,,,,,
183,False,5400.0,"Action,Adventure,Indie","Single-player,Local Co-op,Shared/Split Screen,...","Pixel Graphics,Rogue-like,Action,Dungeon Crawl...","Jul 21, 2018","15 May, 2018",4526.0,480.0,67,False,1.560587,1.701319,1.418910,1.708439
229,False,6800.0,"Action,Indie,Strategy","Single-player,Shared/Split Screen,Steam Achiev...","Strategy,Indie,Action,2D,Physics,Sandbox,Pixel...","Feb 2, 2018","28 Sep, 2012",1132.0,284.0,1953,False,1.560587,1.874730,2.520969,3.658203
252,False,4700.0,"Action,Adventure,Casual,Indie","Single-player,Steam Achievements,Steam Trading...","Action,Adventure,Indie,Casual,Visual Novel,Fem...","Mar 17, 2018","8 Aug, 2017",175.0,25.0,221,False,1.560587,2.976360,3.956501,1.708439
6,False,3600.0,"Action,Indie","Single-player,Steam Achievements,Steam Trading...","Great Soundtrack,Action,Violent,Indie,Top-Down...","Feb 16, 2018","23 Oct, 2012",39776.0,1035.0,1942,False,1.560587,1.874730,2.362442,3.658203
3,False,9900.0,"Action,RPG","Single-player,Multi-player,Steam Achievements,...","Medieval,RPG,Open World,Strategy,Sandbox,Actio...","Nov 28, 2016","31 Mar, 2010",63078.0,1746.0,2434,False,3.077594,1.781419,2.836507,5.265664
310,False,7800.0,"Indie,RPG,Strategy","Single-player,Steam Achievements,Steam Trading...","Turn-Based Combat,Dark Fantasy,Dungeon Crawler...","Feb 20, 2019","19 Jan, 2016",41141.0,6291.0,1128,False,0.802063,1.874730,1.829592,0.813307
239,False,3100.0,"Adventure,Indie","Single-player,Steam Achievements,Steam Trading...","Adventure,Indie,Puzzle,Point & Click","Apr 21, 2018","14 Mar, 2017",50.0,5.0,403,False,1.560587,0.911780,1.418910,1.708439
288,False,6400.0,"Action,RPG","Single-player,Partial Controller Support","FPS,Atmospheric,Action,Story Rich,Singleplayer...","Mar 28, 2018","21 Aug, 2007",15087.0,932.0,3872,False,1.560587,2.976360,2.836507,0.813307
312,False,6999900.0,Action,"Single-player,Multi-player,Co-op,Steam Achieve...","Zombies,Co-op,FPS,Multiplayer,Shooter,Action,O...","Mar 10, 2017","16 Nov, 2009",166310.0,5827.0,2671,False,6.569858,2.976360,2.607755,3.658203


In [19]:
multi_columns = ['genres', 'categories', 'tags']
#multi_columns = ['genres', 'categories']
genres_dummy = X_train['genres'].str.get_dummies(",").add_prefix("genres_")
categories_dummy = X_train['categories'].str.get_dummies(",").add_prefix("categories_")
tags_dummy = X_train['tags'].str.get_dummies(",").add_prefix("tags_")

X_train = X_train.drop(columns=multi_columns)
X_train = pd.concat([X_train, genres_dummy, categories_dummy, tags_dummy], axis=1)
#X_train = pd.concat([X_train, genres_dummy, categories_dummy], axis=1)

multi_choice_dummies = [col for col in X_train 
               if "_" in col 
               and col.split("_")[0] in multi_columns]
pd.set_option('display.max_columns', 500)
X_train.head(100)



processed_columns = list(X_train.columns[:])

test_genres_dummy = Test['genres'].str.get_dummies(",").add_prefix("genres_")
test_categories_dummy = Test['categories'].str.get_dummies(",").add_prefix("categories_")
test_tags_dummy = Test['tags'].str.get_dummies(",").add_prefix("tags_")


Test = Test.drop(columns=multi_columns)
Test = pd.concat([Test, test_genres_dummy, test_categories_dummy, test_tags_dummy], axis=1)
#Test = pd.concat([Test, genres_dummy, categories_dummy], axis=1)

# train里面没有的column，都drop掉
for col in Test.columns:
    if ("_" in col) and (col.split("_")[0] in multi_columns) and col not in multi_choice_dummies:
#         print("Removing additional feature {}".format(col))
        Test.drop(col, axis=1, inplace=True)
# train里有但是val里面没有的column，都填0
for col in multi_choice_dummies:
    if col not in Test.columns:
        Test[col] = 0
        
Test = Test[processed_columns]



In [20]:
Test

,is_free,price,purchase_date,release_date,total_positive_reviews,total_negative_reviews,datediff,prepurchase,purchase_date_year_mean,purchase_date_month_mean,purchase_date_day_mean,purchase_date_weekday_mean,genres_Action,genres_Adventure,genres_Animation & Modeling,genres_Audio Production,genres_Casual,genres_Design & Illustration,genres_Early Access,genres_Free to Play,genres_Gore,genres_Indie,genres_Massively Multiplayer,genres_Nudity,genres_RPG,genres_Racing,genres_Sexual Content,genres_Simulation,genres_Sports,genres_Strategy,genres_Utilities,genres_Violent,categories_Captions available,categories_Co-op,categories_Commentary available,categories_Cross-Platform Multiplayer,categories_Full controller support,categories_In-App Purchases,categories_Includes Source SDK,categories_Includes level editor,categories_Local Co-op,categories_Local Multi-Player,categories_MMO,categories_Multi-player,categories_Online Co-op,categories_Online Multi-Player,categories_Partial Controller Support,categories_Remote Play on Phone,categories_Remote Play on TV,categories_Remote Play on Tablet,categories_Shared/Split Screen,categories_Single-player,categories_Stats,categories_Steam Achievements,categories_Steam Cloud,categories_Steam Leaderboards,categories_Steam Trading Cards,categories_Steam Workshop,categories_SteamVR Collectibles,categories_VR Support,categories_Valve Anti-Cheat enabled,tags_1980s,tags_1990's,tags_2.5D,tags_2D,tags_3D,tags_3D Platformer,tags_3D Vision,tags_4 Player Local,tags_4X,tags_ATV,tags_Action,tags_Action RPG,tags_Action-Adventure,tags_Addictive,tags_Adventure,tags_Aliens,tags_Alternate History,tags_America,tags_Animation & Modeling,tags_Anime,tags_Arcade,tags_Arena Shooter,tags_Artificial Intelligence,tags_Assassin,tags_Atmospheric,tags_Audio Production,tags_Automation,tags_Base Building,tags_Based On A Novel,tags_Batman,tags_Battle Royale,tags_Beat 'em up,tags_Beautiful,tags_Benchmark,tags_Bikes,tags_Blood,tags_Board Game,tags_Building,tags_Bullet Hell,tags_Bullet Time,tags_CRPG,tags_Capitalism,tags_Card Game,tags_Cartoon,tags_Cartoony,tags_Casual,tags_Cats,tags_Character Action Game,tags_Character Customization,tags_Chess,tags_Choices Matter,tags_Choose Your Own Adventure,tags_Cinematic,tags_City Builder,tags_Classic,tags_Clicker,tags_Co-op,tags_Co-op Campaign,tags_Colorful,tags_Comedy,tags_Comic Book,tags_Competitive,tags_Conspiracy,tags_Controller,tags_Crafting,tags_Crime,tags_Crowdfunded,tags_Cult Classic,tags_Cute,tags_Cyberpunk,tags_Dark,tags_Dark Comedy,tags_Dark Fantasy,tags_Dark Humor,tags_Dating Sim,tags_Demons,tags_Design & Illustration,tags_Destruction,tags_Detective,tags_Difficult,tags_Dinosaurs,tags_Diplomacy,tags_Documentary,tags_Dragons,tags_Drama,tags_Driving,tags_Dungeon Crawler,tags_Dungeons & Dragons,tags_Dynamic Narration,tags_Dystopian,tags_Early Access,tags_Economy,tags_Education,tags_Emotional,tags_Epic,tags_Episodic,tags_Experience,tags_Experimental,tags_Exploration,tags_FMV,tags_FPS,tags_Family Friendly,tags_Fantasy,tags_Fast-Paced,tags_Female Protagonist,tags_Fighting,tags_First-Person,tags_Flight,tags_Free to Play,tags_Funny,tags_Futuristic,tags_Game Development,tags_Games Workshop,tags_God Game,tags_Gore,tags_Gothic,tags_Grand Strategy,tags_Great Soundtrack,tags_Gun Customization,tags_Hack and Slash,tags_Hacking,tags_Hand-drawn,tags_Heist,tags_Hex Grid,tags_Hidden Object,tags_Historical,tags_Horror,tags_Horses,tags_Hunting,tags_Illuminati,tags_Immersive Sim,tags_Indie,tags_Intentionally Awkward Controls,tags_Interactive Fiction,tags_Inventory Management,tags_Investigation,tags_Isometric,tags_JRPG,tags_Kickstarter,tags_LGBTQ+,tags_Lara Croft,tags_Level Editor,tags_Linear,tags_Local Co-Op,tags_Local Multiplayer,tags_Logic,tags_Loot,tags_Lovecraftian,tags_MMORPG,tags_MOBA,tags_Magic,tags_Management,tags_Mars,tags_Martial Arts,tags_Massively Multiplayer,tags_Masterpiece,tags_Mature,tags_Mechs,tags_Medieval,tags_Memes,tags_Metroidvania,tags_Military,tags_Minigames,tags_Minimalist,tags_Mod,tags

In [21]:
X_train['playtime_forever'] = Train['playtime_forever']

In [22]:
abc =X_train[Train['playtime_forever'] > 10]

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [23]:
corrmat = abc.corr()
high = corrmat[(corrmat['playtime_forever']>0.4)|(corrmat['playtime_forever']<-0.4)]
cor_target = abs(corrmat["playtime_forever"])
#Selecting highly correlated features
relevant_features = cor_target[cor_target>0.4]
relevant_features


prepurchase                              0.412996
purchase_date_year_mean                  0.511593
purchase_date_month_mean                 0.711579
purchase_date_day_mean                   0.709192
purchase_date_weekday_mean               0.505069
genres_Action                            0.495214
categories_Cross-Platform Multiplayer    0.604759
categories_Includes level editor         0.435755
categories_Multi-player                  0.476485
categories_Online Multi-Player           0.437389
categories_Steam Achievements            0.402208
categories_Steam Workshop                0.661087
tags_Crafting                            0.426529
tags_Party-Based RPG                     0.435755
tags_Strategy                            0.444282
tags_Turn-Based Tactics                  0.435755
playtime_forever                         1.000000
Name: playtime_forever, dtype: float64

In [24]:
corrmat = X_train.corr()
high = corrmat[(corrmat['playtime_forever']>0.15)|(corrmat['playtime_forever']<-0.15)]
high


,is_free,price,total_positive_reviews,total_negative_reviews,datediff,prepurchase,purchase_date_year_mean,purchase_date_month_mean,purchase_date_day_mean,purchase_date_weekday_mean,genres_Action,genres_Adventure,genres_Animation & Modeling,genres_Audio Production,genres_Casual,genres_Design & Illustration,genres_Early Access,genres_Free to Play,genres_Gore,genres_Indie,genres_Massively Multiplayer,genres_Nudity,genres_RPG,genres_Racing,genres_Sexual Content,genres_Simulation,genres_Sports,genres_Strategy,genres_Utilities,genres_Violent,categories_Captions available,categories_Co-op,categories_Commentary available,categories_Cross-Platform Multiplayer,categories_Full controller support,categories_In-App Purchases,categories_Includes Source SDK,categories_Includes level editor,categories_Local Co-op,categories_Local Multi-Player,categories_MMO,categories_Multi-player,categories_Online Co-op,categories_Online Multi-Player,categories_Partial Controller Support,categories_Remote Play on Phone,categories_Remote Play on TV,categories_Remote Play on Tablet,categories_Shared/Split Screen,categories_Single-player,categories_Stats,categories_Steam Achievements,categories_Steam Cloud,categories_Steam Leaderboards,categories_Steam Trading Cards,categories_Steam Workshop,categories_SteamVR Collectibles,categories_VR Support,categories_Valve Anti-Cheat enabled,tags_1980s,tags_1990's,tags_2.5D,tags_2D,tags_3D,tags_3D Platformer,tags_3D Vision,tags_4 Player Local,tags_4X,tags_ATV,tags_Action,tags_Action RPG,tags_Action-Adventure,tags_Addictive,tags_Adventure,tags_Aliens,tags_Alternate History,tags_America,tags_Animation & Modeling,tags_Anime,tags_Arcade,tags_Arena Shooter,tags_Artificial Intelligence,tags_Assassin,tags_Atmospheric,tags_Audio Production,tags_Automation,tags_Base Building,tags_Based On A Novel,tags_Batman,tags_Battle Royale,tags_Beat 'em up,tags_Beautiful,tags_Benchmark,tags_Bikes,tags_Blood,tags_Board Game,tags_Building,tags_Bullet Hell,tags_Bullet Time,tags_CRPG,tags_Capitalism,tags_Card Game,tags_Cartoon,tags_Cartoony,tags_Casual,tags_Cats,tags_Character Action Game,tags_Character Customization,tags_Chess,tags_Choices Matter,tags_Choose Your Own Adventure,tags_Cinematic,tags_City Builder,tags_Classic,tags_Clicker,tags_Co-op,tags_Co-op Campaign,tags_Colorful,tags_Comedy,tags_Comic Book,tags_Competitive,tags_Conspiracy,tags_Controller,tags_Crafting,tags_Crime,tags_Crowdfunded,tags_Cult Classic,tags_Cute,tags_Cyberpunk,tags_Dark,tags_Dark Comedy,tags_Dark Fantasy,tags_Dark Humor,tags_Dating Sim,tags_Demons,tags_Design & Illustration,tags_Destruction,tags_Detective,tags_Difficult,tags_Dinosaurs,tags_Diplomacy,tags_Documentary,tags_Dragons,tags_Drama,tags_Driving,tags_Dungeon Crawler,tags_Dungeons & Dragons,tags_Dynamic Narration,tags_Dystopian,tags_Early Access,tags_Economy,tags_Education,tags_Emotional,tags_Epic,tags_Episodic,tags_Experience,tags_Experimental,tags_Exploration,tags_FMV,tags_FPS,tags_Family Friendly,tags_Fantasy,tags_Fast-Paced,tags_Female Protagonist,tags_Fighting,tags_First-Person,tags_Flight,tags_Free to Play,tags_Funny,tags_Futuristic,tags_Game Development,tags_Games Workshop,tags_God Game,tags_Gore,tags_Gothic,tags_Grand Strategy,tags_Great Soundtrack,tags_Gun Customization,tags_Hack and Slash,tags_Hacking,tags_Hand-drawn,tags_Heist,tags_Hex Grid,tags_Hidden Object,tags_Historical,tags_Horror,tags_Horses,tags_Hunting,tags_Illuminati,tags_Immersive Sim,tags_Indie,tags_Intentionally Awkward Controls,tags_Interactive Fiction,tags_Inventory Management,tags_Investigation,tags_Isometric,tags_JRPG,tags_Kickstarter,tags_LGBTQ+,tags_Lara Croft,tags_Level Editor,tags_Linear,tags_Local Co-Op,tags_Local Multiplayer,tags_Logic,tags_Loot,tags_Lovecraftian,tags_MMORPG,tags_MOBA,tags_Magic,tags_Management,tags_Mars,tags_Martial Arts,tags_Massively Multiplayer,tags_Masterpiece,tags_Mature,tags_Mechs,tags_Medieval,tags_Memes,tags_Metroidvania,tags_Military,tags_Minigames,tags_Minimalist,tags_Mod,tags_Moddable,tags_Motocross,ta

In [25]:

cor_target = abs(corrmat["playtime_forever"])
#Selecting highly correlated features
relevant_features = cor_target[cor_target>0.15]
relevant_features


total_positive_reviews        0.394534
total_negative_reviews        0.392982
datediff                      0.161994
prepurchase                   0.195362
purchase_date_year_mean       0.213902
purchase_date_month_mean      0.220399
purchase_date_day_mean        0.367651
purchase_date_weekday_mean    0.220905
categories_Co-op              0.185111
categories_Multi-player       0.167636
categories_Steam Workshop     0.164782
tags_Battle Royale            0.160215
tags_Co-op                    0.223856
tags_Crafting                 0.213480
tags_Multiplayer              0.173790
tags_Online Co-Op             0.193659
tags_Open World               0.154852
tags_PvP                      0.207030
tags_Rogue-like               0.170366
tags_Survival                 0.173868
tags_Touch-Friendly           0.181687
tags_Turn-Based Combat        0.180914
playtime_forever              1.000000
Name: playtime_forever, dtype: float64

In [26]:
print(X_train[["tags_Online Co-Op","tags_Co-op"]].corr())
print(X_train[["tags_Multiplayer","categories_Multi-player"]].corr())

                   tags_Online Co-Op  tags_Co-op
tags_Online Co-Op           1.000000    0.592706
tags_Co-op                  0.592706    1.000000
                         tags_Multiplayer  categories_Multi-player
tags_Multiplayer                 1.000000                 0.692532
categories_Multi-player          0.692532                 1.000000


In [27]:
X_train = X_train.drop(['playtime_forever'],axis = 1)

In [28]:
X_train =X_train.drop(['purchase_date','release_date',],axis = 1)
TestX =Test.drop(['purchase_date','release_date'],axis = 1)

In [29]:
removecol = [cname for cname in X_train.columns if ((X_train[cname]==0).sum()> 350 and cname!='is_free' and cname!='prepurchase') ]


In [30]:
removecol

['genres_Animation & Modeling',
 'genres_Audio Production',
 'genres_Design & Illustration',
 'genres_Nudity',
 'genres_Racing',
 'genres_Sexual Content',
 'genres_Sports',
 'genres_Utilities',
 'categories_Includes Source SDK',
 'tags_1980s',
 'tags_3D',
 'tags_ATV',
 'tags_America',
 'tags_Animation & Modeling',
 'tags_Arena Shooter',
 'tags_Artificial Intelligence',
 'tags_Assassin',
 'tags_Audio Production',
 'tags_Automation',
 'tags_Based On A Novel',
 'tags_Batman',
 'tags_Battle Royale',
 'tags_Benchmark',
 'tags_Bikes',
 'tags_Blood',
 'tags_Bullet Hell',
 'tags_Bullet Time',
 'tags_Capitalism',
 'tags_Cartoon',
 'tags_Cartoony',
 'tags_Cats',
 'tags_Chess',
 'tags_Clicker',
 'tags_Co-op Campaign',
 'tags_Comic Book',
 'tags_Conspiracy',
 'tags_Dark Comedy',
 'tags_Dating Sim',
 'tags_Demons',
 'tags_Design & Illustration',
 'tags_Dinosaurs',
 'tags_Diplomacy',
 'tags_Documentary',
 'tags_Driving',
 'tags_Dungeons & Dragons',
 'tags_Dynamic Narration',
 'tags_Emotional',
 'tag

In [31]:
TestX['total_positive_reviews'].fillna(TestX['total_positive_reviews'].mean(), inplace=True)
TestX['total_negative_reviews'].fillna(TestX['total_negative_reviews'].mean(), inplace=True)
TestX['is_free'].fillna(False, inplace=True)
TestX['prepurchase'].fillna(False, inplace=True)
TestX['price'].fillna(TestX['price'].mean(), inplace=True)
TestX['purchase_date_year_mean'].fillna(TestX['purchase_date_year_mean'].mean(), inplace=True)
TestX['purchase_date_month_mean'].fillna(TestX['purchase_date_month_mean'].mean(), inplace=True)
TestX['purchase_date_day_mean'].fillna(TestX['purchase_date_day_mean'].mean(), inplace=True)
TestX['purchase_date_weekday_mean'].fillna(TestX['purchase_date_weekday_mean'].mean(), inplace=True)
TestX['datediff'].fillna(TestX['datediff'].mean(), inplace=True)



In [32]:
X_T = X_train[['genres_Action','categories_Includes level editor','categories_Cross-Platform Multiplayer','total_positive_reviews','purchase_date_weekday_mean','prepurchase','purchase_date_year_mean','purchase_date_month_mean','categories_Steam Workshop','tags_Co-op','tags_Online Co-Op','tags_Crafting','tags_Multiplayer','tags_Open World','tags_PvP','tags_Rogue-like','tags_Survival','tags_Touch-Friendly','tags_Turn-Based Combat','tags_Turn-Based Tactics']]

#X_T = X_train['total_positive_reviews','datediff']
# datediff                      0.161994
# prepurchase                   0.195362
# purchase_date_year_mean       0.213902
# purchase_date_month_mean      0.220399
# purchase_date_day_mean        0.367651
# purchase_date_weekday_mean    0.220905
# categories_Co-op              0.185111
# categories_Multi-player       0.167636
# categories_Steam Workshop     0.164782
# tags_Battle Royale            0.160215
# tags_Co-op                    0.223856
# tags_Crafting                 0.213480
# tags_Multiplayer              0.173790
# tags_Online Co-Op             0.193659
# tags_Open World               0.154852
# tags_PvP                      0.207030
# tags_Rogue-like               0.170366
# tags_Survival                 0.173868
# tags_Touch-Friendly           0.181687
# tags_Turn-Based Combat        0.180914

In [33]:
#X_T = preprocessing.normalize(X_T)
X_T = preprocessing.scale(X_T)

In [34]:
import xgboost as xgb
#my_model = xgb.XGBRegressor()
#my_model = xgb.XGBRegressor(max_depth=4, learning_rate=0.1, n_estimators=100, verbosity=1,tree_method='auto', reg_alpha=0.05, reg_lambda=0.05)
my_model2 = xgb.XGBClassifier()
#my_model = xgb.XGBRegressor(n_estimators=500, learning_rate=0.05)
#my_model = xgb.XGBClassifier(n_estimators=500, learning_rate=0.05)
my_model = xgb.XGBRegressor(max_depth=2,gamma=2,eta=0.8,reg_alpha=0.5,reg_lambda=0.5)
# my_model.fit(X_train, y_train, 
#              early_stopping_rounds=5, 
#              eval_set=[(X_valid, y_valid)], 
#              verbose=False)
#my_model.fit(X_train, Y_train, eval_metric='merror', early_stopping_rounds=5, eval_set=[(X_train, Y_train)], verbose=True)

from xgboost import plot_importance
import matplotlib.pyplot as plt

my_model.fit(X_T, Y_train)

# Test = Test['playtime_forever']
# Test.to_csv('submission.csv')
# plot feature importance
# plot_importance(my_model)
# pyplot.show()

[14:12:17] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


//anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
//anaconda3/lib/python3.7/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, eta=0.8, gamma=2,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=2, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0.5, reg_lambda=0.5, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [35]:
from sklearn import neighbors
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge

knn = neighbors.KNeighborsRegressor(n_neighbors=10, weights='uniform', algorithm='auto', leaf_size=30, p=2, metric='minkowski')
#knn2 = neighbors.KNeighborsRegressor(n_neighbors=25, weights='distance', algorithm='auto', leaf_size=30, p=2, metric='minkowski')
#knn3 = neighbors.KNeighborsRegressor(n_neighbors=20, weights='uniform', algorithm='auto', leaf_size=30, p=2, metric='minkowski')



#knn.fit(X_T, Y_train)

#knn2.fit(X_train, Y_train)

regr = linear_model.LinearRegression()
#regr.fit(X_T, Y_train)

rf =RandomForestRegressor()
#rf.fit(X_T, Y_train)

reg = linear_model.Lasso(alpha=0.1)
regri=Ridge(alpha=1.0)


In [36]:
#lgr = 

In [37]:
from sklearn.model_selection import cross_val_score


# Use cross_val_score function
# We are passing the entirety of X and y, not X_train or y_train, it takes care of splitting the dat
# cv=10 for 10 folds
# scoring='accuracy' for evaluation metric - althought they are many
scores = cross_val_score(knn, X_T, Y_train, cv=3, scoring='neg_mean_squared_error')
scores2 = cross_val_score(rf, X_T, Y_train, cv=3, scoring='neg_mean_squared_error')
scores3=cross_val_score(my_model, X_T, Y_train, cv=3, scoring='neg_mean_squared_error')
scores4=cross_val_score(reg, X_T, Y_train, cv=3, scoring='neg_mean_squared_error')
scores5=cross_val_score(my_model2, X_T, Y2_train, cv=3, scoring='accuracy')
print(np.mean(np.sqrt((abs(scores)))))
print(np.mean(np.sqrt((abs(scores2)))))
print(np.mean(np.sqrt((abs(scores3)))))
print(np.mean(np.sqrt((abs(scores4)))))
print(scores5)

//anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
//anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
//anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
//anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
//anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed i

[14:12:17] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:12:17] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:12:17] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
10.614488794106064
10.187992429804023
10.771071409504879
10.708346891097433
[0.57983193 0.65546218 0.69230769]


In [38]:
TestX
T_X=TestX[['genres_Action','categories_Includes level editor','categories_Cross-Platform Multiplayer','total_positive_reviews','purchase_date_weekday_mean','datediff','purchase_date_year_mean','purchase_date_month_mean','categories_Steam Workshop','tags_Co-op','tags_Online Co-Op','tags_Crafting','tags_Multiplayer','tags_Open World','tags_PvP','tags_Rogue-like','tags_Survival','tags_Touch-Friendly','tags_Turn-Based Combat','tags_Turn-Based Tactics']]


In [39]:
#Test['playtime_forever']= my_model.predict(TestX)
#rf.fit(X_train, Y_train)
reg.fit(X_T, Y_train)
my_model2.fit(X_T, Y2_train)
T_XN = preprocessing.scale(T_X)
Test['playtime_forever']= (reg.predict(T_XN)*my_model2.predict(T_XN))
Test['playtime_forever'] = Test['playtime_forever'].apply(lambda x: 0 if x < 0 else x)

result = Test.loc[:, ['playtime_forever']]
result.to_csv('submission.csv')

In [40]:
Test['playtime_forever'].value_counts()

0.000000     39
0.876069      1
12.398809     1
6.093881      1
0.836518      1
4.970447      1
12.285847     1
0.495565      1
0.192477      1
3.075720      1
1.619243      1
4.256090      1
8.962570      1
3.006340      1
1.047319      1
6.639067      1
2.686875      1
4.932290      1
2.810032      1
1.596524      1
0.189715      1
7.018217      1
1.222058      1
4.520855      1
9.685405      1
2.449924      1
0.888697      1
3.425225      1
21.581520     1
16.844188     1
2.082842      1
12.997467     1
12.533787     1
5.640230      1
18.152882     1
4.742391      1
0.360483      1
1.080294      1
3.008971      1
3.889860      1
6.709768      1
4.240368      1
12.673006     1
1.219601      1
17.016830     1
13.868613     1
23.263524     1
2.112715      1
22.166913     1
4.589901      1
3.592607      1
0.062717      1
Name: playtime_forever, dtype: int64

In [41]:
Test['playtime_forever'].describe()

count    90.000000
mean      3.584592
std       5.677956
min       0.000000
25%       0.000000
50%       0.856293
75%       4.572640
max      23.263524
Name: playtime_forever, dtype: float64

In [42]:
result

,playtime_forever
id,
0,1.222058
1,0.000000
2,-0.000000
3,0.000000
4,13.868613
5,0.000000
6,0.000000
7,0.000000
8,0.000000
